# Setup

In [1]:
install_load <- function (package1, ...)  {   

   # convert arguments to vector
   packages <- c(package1, ...)

   # start loop to determine if each package is installed
   for(package in packages){

       # if package is installed locally, load
       if(package %in% rownames(installed.packages()))
          do.call('library', list(package))

       # if package is not installed locally, download, then load
       else {
          install.packages(package)
          do.call("library", list(package))
       }
   } 
}

# Section 9.2

In [2]:
install_load("mlr", "tidyverse", "FSelector")

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
also installing the dependencies ‘ParamHelpers’, ‘BBmisc’, ‘data.table’, ‘parallelMap’

Loading required package: ParamHelpers
'mlr' is in maintenance mode since July 2019. Future development
efforts will go into its successor 'mlr3' (<https://mlr3.mlr-org.com>).
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.0     ✔ purrr   0.3.3
✔ tibble  2.1.3     ✔ dplyr   0.8.1
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


## Section 9.2.1

### Listing 9.1

In [5]:
data(Ozone, package = "mlbench")
ozoneTib <- as_tibble(Ozone)
names(ozoneTib) <- c("Month", "Date", "Day", "Ozone", "Press_height",
                     "Wind", "Humid", "Temp_Sand", "Temp_Monte",
                     "Inv_height", "Press_grad", "Inv_temp", "Visib")
head(ozoneTib)

Month,Date,Day,Ozone,Press_height,Wind,Humid,Temp_Sand,Temp_Monte,Inv_height,Press_grad,Inv_temp,Visib
<fct>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,4,3,5480,8,20,NA,NA,5000,-15,30.56,200
1,2,5,3,5660,6,NA,38,NA,NA,-14,NA,300
1,3,6,3,5710,4,28,40,NA,2693,-25,47.66,250
1,4,7,5,5700,3,37,45,NA,590,-24,55.04,100
1,5,1,5,5760,3,51,54,45.32,1450,25,57.02,60
1,6,2,6,5720,4,69,35,49.64,1568,15,53.78,60


In [7]:
# missing data
map_dbl(ozoneTib, ~sum(is.na(.)))

Month         Date          Day        Ozone Press_height         Wind 
           0            0            0            5           12            0 
       Humid    Temp_Sand   Temp_Monte   Inv_height   Press_grad     Inv_temp 
          15            2          139           15            1           14 
       Visib 
           0

### Listing 9.2

In [8]:
ozoneClean <- mutate_all(ozoneTib, as.numeric) %>%
    filter(is.na(Ozone) == FALSE)

head(ozoneClean)

Month,Date,Day,Ozone,Press_height,Wind,Humid,Temp_Sand,Temp_Monte,Inv_height,Press_grad,Inv_temp,Visib
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,4,3,5480,8,20,NA,NA,5000,-15,30.56,200
1,2,5,3,5660,6,NA,38,NA,NA,-14,NA,300
1,3,6,3,5710,4,28,40,NA,2693,-25,47.66,250
1,4,7,5,5700,3,37,45,NA,590,-24,55.04,100
1,5,1,5,5760,3,51,54,45.32,1450,25,57.02,60
1,6,2,6,5720,4,69,35,49.64,1568,15,53.78,60


### Listing 9.3

In [10]:
ozoneUntidy <- gather(ozoneClean, key = "Variable", value = "Value", -Ozone)

options(repr.plot.width=15, repr.plot.height=8)
ggplot(ozoneUntidy, aes(Value, Ozone)) +
    facet_wrap(~ Variable, scale = "free_x") + 
    geom_point() +
    geom_smooth() +
    geom_smooth(method = "lm", col = "red") + 
    theme_bw()

`geom_smooth()` using method = 'loess' and formula 'y ~ x'
Warning message:
“Removed 196 rows containing non-finite values (stat_smooth).”Warning message:
“Removed 196 rows containing non-finite values (stat_smooth).”Warning message:
“Removed 196 rows containing missing values (geom_point).”

## Section 9.2.2

### Listing 9.4

In [13]:
imputeMethod <- imputeLearner("regr.rpart")
ozoneImp <- impute(as.data.frame(ozoneClean), classes = list(numeric = imputeMethod))

### Listing 9.5

In [14]:
ozoneTask <- makeRegrTask(data = ozoneImp$data, target = "Ozone")
lin <- makeLearner("regr.lm")